In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import preprocessing as pp
import manage_data as md
%matplotlib osx
plt.ion()

In [2]:
coil = 10
dfi = md.import_dfi()
df = md.import_data(coil)
fs = 10000
signal = df['micro'].values
rms = pp.fast_rms(signal)
a = 0
startS, endS = dfi.startS[coil], dfi.endS[coil]

a, b = 90,180 
signal = signal[a*fs: b*fs]
rms = rms[a*fs: b*fs]

In [3]:
import processing as pc

In [4]:
import pyeemd
import scipy.signal

In [5]:
emd = pyeemd.emd(signal/rms, num_imfs=1)

In [6]:
emd.shape[-1]

2515300

In [7]:
emd.shape = signal.shape
emd.shape[-1]

2515300

In [8]:
hht = scipy.signal.hilbert(emd)

In [9]:
amp0 = np.abs(hht)

In [10]:
t, corr = pc.rolling_correlation_convolution(amp0, fs, beginning=a)

10000 0 40000 40000
50000 40000 80000 40000
90000 80000 120000 40000
130000 120000 160000 40000
170000 160000 200000 40000
210000 200000 240000 40000
250000 240000 280000 40000
290000 280000 320000 40000
330000 320000 360000 40000
370000 360000 400000 40000
410000 400000 440000 40000
450000 440000 480000 40000
490000 480000 520000 40000
530000 520000 560000 40000
570000 560000 600000 40000
610000 600000 640000 40000
650000 640000 680000 40000
690000 680000 720000 40000
730000 720000 760000 40000
770000 760000 800000 40000
810000 800000 840000 40000
850000 840000 880000 40000
890000 880000 920000 40000
930000 920000 960000 40000
970000 960000 1000000 40000
1010000 1000000 1040000 40000
1050000 1040000 1080000 40000
1090000 1080000 1120000 40000
1130000 1120000 1160000 40000
1170000 1160000 1200000 40000
1210000 1200000 1240000 40000
1250000 1240000 1280000 40000
1290000 1280000 1320000 40000
1330000 1320000 1360000 40000
1370000 1360000 1400000 40000
1410000 1400000 1440000 40000
145000

In [11]:
plt.figure()
for time, correlation in zip(t, corr):
    plt.plot(time, correlation)

In [12]:
plt.figure()
for correlation in corr:
    dt = 1./fs
    fft = np.fft.fft(correlation)
    freq = np.fft.fftfreq(len(correlation), 1./fs)
    nTot = fft.shape[0]
    nPoints_to_2Hz = int(nTot*dt*2)
    nPoints_to_10Hz = int(nTot*dt*10)
    idx = np.arange(nPoints_to_2Hz,nPoints_to_10Hz)
    #plt.figure()
    plt.plot(freq[idx], np.abs(fft[idx]))

In [13]:
freq_list, fft_list = pc.fft_of_correlation(corr, fs)

In [14]:
def peak(freq, fft):
    imax = np.argmax(fft)
    xmax = freq[imax]
    ymax = fft[imax]
    return list([xmax, ymax])

In [15]:
peaks = []
for freq, fft in zip(freq_list, fft_list):
    peaks.append(peak(freq, fft))
peaks

[[1.6001920230427649, 158.76171894343292],
 [2.8003360403248383, 24.88167860518805],
 [1.6001920230427649, 77.200228419671149],
 [1.6001920230427649, 40.058580040145799],
 [2.4002880345641473, 24.349977538015875],
 [1.6001920230427649, 83.435291759766827],
 [1.6001920230427649, 44.891472094841433],
 [1.6001920230427649, 12.70838003684743],
 [2.0002400288034563, 294.98185957663856],
 [2.8003360403248383, 27.928363208400942],
 [2.0002400288034563, 12.719424799740047],
 [2.0002400288034563, 23.090456128672585],
 [2.8003360403248383, 28.538463952457754],
 [1.6001920230427649, 76.391735120604395],
 [2.4002880345641473, 13.579365860903247],
 [1.6001920230427649, 123.63443787397432],
 [2.8003360403248383, 52.660853987436326],
 [1.6001920230427649, 22.275222286122894],
 [3.2003840460855297, 19.471028536284379],
 [6.000720086410368, 7.4746045236747785],
 [4.4005280633676032, 8.562110984534943],
 [3.6004320518462212, 23.291696938659943],
 [8.0009601152138252, 13.667787494258086],
 [2.40028803456

In [16]:
dfi.iloc[coil]

coil_number                                    47465785
datum_post                          2014-11-25 00:00:00
real_date                           2014-11-25 00:00:00
time                         2021-07-04 15:54:59.995000
length                                             2710
width                                              1411
thickness                                           0.9
weight                                           27.051
oil                                                 269
steelgrade                                       K04AA0
inspection                                            1
sticking                                              0
startS                                              NaN
endS                                                NaN
special_case                                          0
comment         not mentioned, normally no stickermarks
sound_file                                            1
duration                                        

In [17]:
time = df['time'].values

In [18]:
decoiler = df['decoiler'].values
coiler = df['coiler'].values

In [19]:
import preprocessing as pp

plt.figure()
sticking = pp.sticking_detection(decoiler-decoiler[-1], 797, 1896)
plt.plot(time, sticking)

In [20]:
sticking = pp.sticking_detection(coiler-coiler[0], startS, endS)
plt.plot(time, sticking/5.)